In [1]:
# Init
import glob
import os
import pickle
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

import multiprocessing
import warnings

# Imports
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)
# For loading data
import concurrent.futures
import gc
import json
import random

# Plotting imports
import altair as alt
import seaborn as sns
from matplotlib.colors import ListedColormap
from numpy import concatenate as c
from numpy import cumsum

In [2]:
# Acc. functions
def returnLast(lst):
    return lst[-1]


def returnGen(lst):
    return lst[1500]


def returnList(HGTGen, lst):
    return lst[HGTGen]


def colorRanges(i):
    # Construct the colormap
    current_palette = sns.color_palette("muted", n_colors=5)
    cmap = ListedColormap(sns.color_palette(current_palette).as_hex())
    # range and stuff
    if i == 0:
        return "0"
    elif i > 0 and i <= 20:
        return "1-20"
    elif i > 20 and i <= 50:
        return "20-50"
    elif i > 50 and i <= 100:
        return "50-100"
    elif i > 100 and i <= 200:
        return "100-200"
    elif i > 200 and i <= 300:
        return "200-300"
    elif i > 300:
        return ">300"


# Since pickle format is not working as intended.
# Insert angry thoughts abouts pickle format here
# A function to open pickle files and convert into json
def pickleToJson(resultDir):
    for file in glob.glob(os.path.join(resultDir, "*_2.pickle")):
        with open(file, "rb") as f:
            # print ("Converting file : " + file)
            gc.disable()
            data = pickle.load(f)
            del data["TEset"]
            del data["NTE"]
            del data["NTI"]
            del data["AvgFit"]
            del data["Freq"]
            gc.enable()
            with open(file + ".json", "w") as fp:
                json.dump(data, fp)
    return 0

In [3]:
current_palette = sns.color_palette("muted", n_colors=5)
cmap = ListedColormap(sns.color_palette(current_palette).as_hex())

In [4]:
# Path to results
resultDir = "/home/siddharth/git/Simulicron/Project_1C/coreSim/"

# Convert them to json
returnValue = pickleToJson(resultDir)

In [5]:
# Read the json files into pandas dataframe
SimulationRecords = []
for filename in glob.glob(os.path.join(resultDir, "*.pickle.json")):
    with open(filename, "r") as file:
        SimulationRecords.append(json.load(file))

SimulationResultsDataFrame = pd.DataFrame(SimulationRecords)

In [6]:
SimulationResultsDataFrame

,State,Generatrion,AvgCopyNum,CopyNumVar,TEfamilyCN,TEfamilyVR,TEfamilyRg,HGTGen,ETA,TRate,Tau,selPen,piRNA,TEpi,FitnessFunction,epistasisCoefficient,TECoreOverlap
0,FLUX,4001,"[0.2, 0.192, 0.217, 0.245, 0.256, 0.252, 0.265...","[0.37199999999999994, 0.273136, 0.323910999999...","{'1': [0.2, 0.192, 0.217, 0.245, 0.256, 0.252,...","{'1': [[0.37199999999999994], [0.273136], [0.3...","{'1': [0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.003, ...",300,0.0,"[0.05, 0.05]",1.0,-0.005,"[849, 850, 851, 852, 853, 854, 855, 856, 857, ...","{'1': 1.162, '2': 1.289}",2,-10.742484,"[0, {'1': 0.0, '2': 0.0}, {'1': 0.0, '2': 0.0}..."
1,FLUX,4001,"[0.2, 0.208, 0.201, 0.21, 0.234, 0.219, 0.231,...","[0.384, 0.342736, 0.2585989999999999, 0.239900...","{'1': [0.2, 0.208, 0.201, 0.21, 0.234, 0.219, ...","{'1': [[0.384], [0.342736], [0.258598999999999...","{'1': [0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.002, ...",300,0.0,"[0.05, 0.05]",1.0,-0.005,"[632, 633, 634, 635, 636, 637, 638, 639, 640, ...","{'1': 0.202, '2': 0.244}",2,-98.728279,"[0, {'1': 0.001, '2': 0.0}, {'1': 0.0, '2': 0...."
2,FLUX,4001,"[0.2, 0.208, 0.224, 0.193, 0.2, 0.216, 0.242, ...","[0.386, 0.362736, 0.33182400000000006, 0.22975...","{'1': [0.2, 0.208, 0.224, 0.193, 0.2, 0.216, 0...","{'1': [[0.386], [0.362736], [0.331824000000000...","{'1': [0.0, 0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0....",300,0.0,"[0.05, 0.05]",1.0,-0.005,"[452, 453, 454, 455, 456, 457, 458, 459, 460, ...","{'1': 1.056, '2': 1.067}",2,-12.893827,"[0, {'1': 0.0, '2': 0.0}, {'1': 0.001, '2': 0...."
3,FLUX,4001,"[0.2, 0.205, 0.17, 0.161, 0.17, 0.195, 0.175, ...","[0.34199999999999997, 0.29897500000000005, 0.2...","{'1': [0.2, 0.205, 0.17, 0.161, 0.17, 0.195, 0...","{'1': [[0.34199999999999997], [0.2989750000000...","{'1': [0.0, 0.0, 0.0, 0.0, 0.001, 0.001, 0.001...",300,0.0,"[0.05, 0.05]",1.0,-0.005,"[133, 134, 135, 136, 137, 138, 139, 140, 141, ...","{'1': 0.383, '2': 0.356}",2,-74.262447,"[0, {'1': 0.0, '2': 0.0}, {'1': 0.0, '2': 0.0}..."
4,FLUX,4001,"[0.2, 0.226, 0.242, 0.258, 0.26, 0.274, 0.25, ...","[0.366, 0.32092400000000004, 0.313436, 0.31943...","{'1': [0.2, 0.226, 0.242, 0.258, 0.26, 0.274, ...","{'1': [[0.366], [0.32092400000000004], [0.3134...","{'1': [0.0, 0.001, 0.001, 0.002, 0.001, 0.0, 0...",300,0.0,"[0.05, 0.05]",1.0,-0.005,"[622, 623, 624, 625, 626, 627, 628, 629, 630, ...","{'1': 0.436, '2': 0.57}",2,-44.078740,"[0, {'1': 0.001, '2': 0.0}, {'1': 0.002, '2': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,FLUX,4001,"[0.2, 0.216, 0.252, 0.232, 0.224, 0.231, 0.245...","[0.328, 0.317344, 0.3404960000000001, 0.294176...","{'1': [0.2, 0.216, 0.252, 0.232, 0.224, 0.231,...","{'1': [[0.328], [0.317344], [0.340496000000000...","{'1': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003, 0....",300,0.0,"[0.05, 0.05]",1.0,-0.005,"[296, 297, 298, 299, 300, 301, 302, 303, 304, ...","{'1': 0.227, '2': 0.257}",2,-97.097513,"[0, {'1': 0.0, '2': 0.0}, {'1': 0.0, '2': 0.0}..."
1496,FLUX,4001,"[0.2, 0.185, 0.226, 0.247, 0.245, 0.256, 0.271...","[0.32399999999999995, 0.260775, 0.300924, 0.29...","{'1': [0.2, 0.185, 0.226, 0.247, 0.245, 0.256,...","{'1': [[0.32399999999999995], [0.260775], [0.3...","{'1': [0.0, 0.001, 0.004, 0.004, 0.004, 0.002,...",300,0.0,"[0.05, 0.05]",1.0,-0.005,"[781, 782, 783, 784, 785, 786, 787, 788, 789, ...","{'1': 2.696, '2': 2.016}",2,0.419421,"[0, {'1': 0.001, '2': 0.0}, {'1': 0.002, '2': ..."
1497,FLUX,4001,"[0.2, 0.248, 0.246, 0.27, 0.276, 0.287, 0.287,...","[0.39399999999999996, 0.508496, 0.403483999999...","{'1': [0.2, 0.248, 0.246, 0.27, 0.276, 0.287, ...","{'1': [[0.39399999999999996], [0.508496], [0.4...","{'1': [0.0, 0.0, 0.0, 0.0, 0.002, 0.008, 0.006...",300,0.0,"[0.05, 0.05]",1.0,-0.005,"[285, 286, 287, 288, 289, 290, 291, 292, 293, ...","{'1': 0.663, '2': 0.494}",2,-37.297609,"[0, {'1': 0.0, '2': 0.0}, {'1': 0.0, '2': 0.0}..."
1498,FLUX,4001,"[0.2, 0.177, 0.186, 0.181, 0.194, 0.213, 0.216...","[0.36199999999999993, 0.285671, 0.261404000000...","{'1': [0.2, 0.177, 0.186, 0.181, 0.194

In [7]:
# Create a copy for inplace handling
resultDataframeC = SimulationResultsDataFrame.copy()

In [8]:
# Perform some data wrangling
resultDataframeC[["TranspositionRate_1", "TranspositionRate_2"]] = pd.DataFrame(
    resultDataframeC.TRate.tolist(), index=resultDataframeC.index
)
resultDataframeC[["TE_1_CN", "TE_2_CN"]] = resultDataframeC["TEfamilyCN"].apply(
    pd.Series
)
resultDataframeC[["TE_1_RG", "TE_2_RG"]] = resultDataframeC["TEfamilyRg"].apply(
    pd.Series
)
resultDataframeC[["TE_1_pi", "TE_2_pi"]] = resultDataframeC["TEpi"].apply(pd.Series)

# Add column for the last acn (average copy number) for each transposon family
resultDataframeC["TE_1_ACN"] = resultDataframeC["TE_1_CN"].apply(
    lambda x: returnLast(x)
)
resultDataframeC["TE_2_ACN"] = resultDataframeC["TE_2_CN"].apply(
    lambda x: returnLast(x)
)

# Add column for copies not present in clusters
resultDataframeC["TE_1_NCN"] = (
    resultDataframeC["TE_1_ACN"] - resultDataframeC["TE_1_pi"]
)
resultDataframeC["TE_2_NCN"] = (
    resultDataframeC["TE_2_ACN"] - resultDataframeC["TE_2_pi"]
)
resultDataframeC["TE_1_NCN_l"] = np.log(
    resultDataframeC["TE_1_ACN"] - resultDataframeC["TE_1_pi"]
)
resultDataframeC["TE_2_NCN_l"] = np.log(
    resultDataframeC["TE_2_ACN"] - resultDataframeC["TE_2_pi"]
)

# Add column for nth generation copy number
resultDataframeC["TE_1_ACN_N"] = resultDataframeC["TE_1_CN"].apply(
    lambda x: returnGen(x)
)
resultDataframeC["TE_2_ACN_N"] = resultDataframeC["TE_2_CN"].apply(
    lambda x: returnGen(x)
)

# Get ranges for adding colors later
resultDataframeC["TE_1_NCN_R"] = resultDataframeC["TE_1_NCN"].apply(
    lambda x: colorRanges(x)
)
resultDataframeC["TE_2_NCN_R"] = resultDataframeC["TE_2_NCN"].apply(
    lambda x: colorRanges(x)
)

# Some wrangling for plotly go
x = resultDataframeC["ETA"]
y = resultDataframeC["selPen"]
TE1ACN = resultDataframeC["TE_1_ACN"]
TE2ACN = resultDataframeC["TE_2_ACN"]

# Convert count to log scale
# resultDataframeC["TE_1_ACN_l"] = np.log(resultDataframeC["TE_1_ACN"])
# resultDataframeC["TE_2_ACN_l"] = np.log(resultDataframeC["TE_2_ACN"])

/home/siddharth/mambaforge/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
# Filter dataframe
# Selecting for simulations with two transposons
resultDataframeC["HGT_TE2_CN"] = resultDataframeC["TE_2_CN"].apply(
    lambda x: returnList(301, x)
)
resultDataframeC = resultDataframeC[resultDataframeC["HGT_TE2_CN"] > 0.0]
resultDataframeC = resultDataframeC[resultDataframeC["epistasisCoefficient"] != 0.0]

In [10]:
# Scatterplot for epi.
TE_1_ACN = resultDataframeC["TE_1_ACN"].tolist()
TE_2_ACN = resultDataframeC["TE_2_ACN"].tolist()
epistasisCoefficient = resultDataframeC["epistasisCoefficient"].tolist()

# Create a DF
plotFrame = pd.DataFrame(
    {
        "TE 2 Copy Number (log)": np.log(TE_1_ACN),
        "TE 1 Copy Number (log)": np.log(TE_2_ACN),
        "Epistasis Coefficient": epistasisCoefficient,
    }
)

# For charting
vmin = min(epistasisCoefficient)
vmax = max(epistasisCoefficient)

/tmp/ipykernel_3502255/2059146847.py:10: RuntimeWarning: divide by zero encountered in log
  "TE 1 Copy Number (log)": np.log(TE_2_ACN),


In [11]:
# In altair
alt.Chart(plotFrame).mark_circle().encode(
    x="Epistasis Coefficient",
    y="TE 2 Copy Number (log)",
    color="TE 1 Copy Number (log)"
).properties(
    height=800,
    width=800,
    title="TE 1 and 2 interaction with different epistasis coefficients at generation 4000",
).configure_circle(
    size=400
).configure_axis(
    labelFontSize=20, titleFontSize=25
).configure_title(
    fontSize=30
).configure_legend(
    labelFontSize=20, titleFontSize=25
)

alt.Chart(...)

In [12]:
# Scatterplot for epi.
TE_1_ACN = resultDataframeC["TE_1_ACN_N"].tolist()
TE_2_ACN = resultDataframeC["TE_2_ACN_N"].tolist()
epistasisCoefficient = resultDataframeC["epistasisCoefficient"].tolist()

# Create a DF
plotFrame = pd.DataFrame(
    {
        "TE 2 Copy Number (log)": np.log(TE_1_ACN),
        "TE 1 Copy Number (log)": np.log(TE_2_ACN),
        "Epistasis Coefficient": epistasisCoefficient,
    }
)

# For charting
vmin = min(epistasisCoefficient)
vmax = max(epistasisCoefficient)

/tmp/ipykernel_3502255/1411591456.py:10: RuntimeWarning: divide by zero encountered in log
  "TE 1 Copy Number (log)": np.log(TE_2_ACN),


In [13]:
# In altair
alt.Chart(plotFrame).mark_circle().encode(
    x="Epistasis Coefficient",
    y="TE 2 Copy Number (log)",
    color="TE 1 Copy Number (log)"
).properties(
    height=800,
    width=800,
    title="TE 1 and 2 interaction with different epistasis coefficients at generation 1500",
).configure_circle(
    size=400
).configure_axis(
    labelFontSize=20, titleFontSize=25
).configure_title(
    fontSize=30
).configure_legend(
    labelFontSize=20, titleFontSize=25
)

alt.Chart(...)